<a href="https://colab.research.google.com/github/talhaanwarch/Offensive-Language-Detection/blob/master/ERNIE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install paddlepaddle-gpu
!wget https://ernie.bj.bcebos.com/ERNIE_Base_en_stable-2.0.0.tar.gz
!gunzip ERNIE_Base_en_stable-2.0.0.tar.gz
!tar -xvf ERNIE_Base_en_stable-2.0.0.tar
!git clone https://github.com/PaddlePaddle/ERNIE.git

     |████████████████████████████████| 258.7MB 64kB/s 
     |████████████████████████████████| 25.2MB 130kB/s 
     |████████████████████████████████| 122kB 64.9MB/s 
  Created wheel for rarfile: filename=rarfile-3.1-cp36-none-any.whl size=24908 sha256=2ad49550addbe89f8472d3677b57d41574be67a7017af8bea086ec7d497d223d
  Stored in directory: /root/.cache/pip/wheels/23/3c/c8/0215b6a5079492eff3be3f545ae0b0c4a66734c35c9e444eac
Successfully built rarfile
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
--2020-03-11 16:40:36--  https://ernie.bj.bcebos.com/ERNIE_Base_en_stable-2.0.0.tar.gz
Resolving ernie.bj.bcebos.com (ernie.bj.bcebos.com)... 103.235.46.61
Connecting to ernie.bj.bcebos.com (ernie.bj.bcebos.com)|103.235.46.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 405413980 (3

## Import Packages

In [0]:
import pandas as pd
import os
import re

## Downloading and Preparing Dataset

In [3]:
train=pd.read_csv('https://raw.githubusercontent.com/talhaanwarch/Offensive-Language-Detection/master/Data/olid-training-v1.0.tsv',sep='\t')
train.head()

,id,tweet,subtask_a,subtask_b,subtask_c
0,86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND
2,16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN
4,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN


In [0]:
mapping = {'OFF': 1, 'NOT': 0}
train['subtask_a']=train['subtask_a'].map(mapping)

In [5]:
train.head()

,id,tweet,subtask_a,subtask_b,subtask_c
0,86426,@USER She should ask a few native Americans wh...,1,UNT,NaN
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,1,TIN,IND
2,16820,Amazon is investigating Chinese employees who ...,0,NaN,NaN
3,62688,"@USER Someone should'veTaken"" this piece of sh...",1,UNT,NaN
4,43605,@USER @USER Obama wanted liberals &amp; illega...,0,NaN,NaN


In [0]:
train['tweet']=train['tweet'].str.replace('@USER', "") 
train['tweet']=train['tweet'].str.replace('\d+', '')
train['tweet']=train['tweet'].str.lower()
train['tweet']=train['tweet'].str.replace('[^\w\s]','')


In [7]:
train=train.iloc[:,1:3]
train.head()

,tweet,subtask_a
0,she should ask a few native americans what th...,1
1,go home youre drunk maga trump url,1
2,amazon is investigating chinese employees who ...,0
3,someone shouldvetaken this piece of shit to a...,1
4,obama wanted liberals amp illegals to move i...,0


In [0]:
from sklearn.model_selection import train_test_split
train,val=train_test_split(train,test_size=0.2)

In [9]:
test_data=pd.read_csv('https://raw.githubusercontent.com/talhaanwarch/Offensive-Language-Detection/master/Data/testset-levela.tsv',sep='\t')
test_label=pd.read_csv('https://raw.githubusercontent.com/talhaanwarch/Offensive-Language-Detection/master/Data/labels-levela.csv',header=None )
test=pd.concat([test_data,test_label],axis=1)
test=test.drop(test.columns[2],axis=1)
test.columns=['id','tweet','subtask_a']
test_id=test.pop('id')
test.head()

,tweet,subtask_a
0,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...,OFF
1,"#ConstitutionDay is revered by Conservatives, ...",NOT
2,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...,NOT
3,#Watching #Boomer getting the news that she is...,NOT
4,#NoPasaran: Unity demo to oppose the far-right...,OFF


In [0]:
mapping = {'OFF': 1, 'NOT': 0}
test['subtask_a']=test['subtask_a'].map(mapping)

In [11]:
print(len(train), len(val), len(test))

10592 2648 860


In [0]:
train.columns=['text_a','label']
val.columns=['text_a','label']
test.columns=['text_a','label']

In [13]:
train.to_csv('train.tsv', index=False, sep='\t')
val.to_csv('dev.tsv', index=False, sep='\t')
test.to_csv('test.tsv', index=False, sep='\t')
print("saved")

saved


## Get environment ready for train and testing

### -create new folder for dataset
### -set path for model parameters and dataset

In [0]:
!mkdir -p 'dataset/SST-2'
!mkdir -p parameters/params
!mv train.tsv dataset/SST-2/
!mv dev.tsv dataset/SST-2/
!mv test.tsv dataset/SST-2/


In [0]:
!mv /content/params parameters/params

In [0]:
!mv dataset/ ERNIE/
!mv parameters/ ERNIE/

## Start Model Training

In [17]:
os.chdir('ERNIE/')
os.environ['TASK_DATA_PATH']='dataset'
os.environ['MODEL_PATH']='parameters/params'
print(os.getcwd())
!sh script/en_glue/ernie_base/SST-2/task.sh

/content/ERNIE
script/en_glue/ernie_base/SST-2/task.sh: 8: script/en_glue/ernie_base/SST-2/task.sh: [[: not found
2020-03-11 16:41:59,217-INFO: -----------  Configuration Arguments -----------
[INFO] 2020-03-11 16:41:59,217 [     args.py:   68]:	-----------  Configuration Arguments -----------
2020-03-11 16:41:59,217-INFO: batch_size: 32
[INFO] 2020-03-11 16:41:59,217 [     args.py:   70]:	batch_size: 32
2020-03-11 16:41:59,217-INFO: checkpoints: ./checkpoints
[INFO] 2020-03-11 16:41:59,217 [     args.py:   70]:	checkpoints: ./checkpoints
2020-03-11 16:41:59,217-INFO: chunk_scheme: IOB
[INFO] 2020-03-11 16:41:59,217 [     args.py:   70]:	chunk_scheme: IOB
2020-03-11 16:41:59,217-INFO: decr_every_n_nan_or_inf: 2
[INFO] 2020-03-11 16:41:59,217 [     args.py:   70]:	decr_every_n_nan_or_inf: 2
2020-03-11 16:41:59,217-INFO: decr_ratio: 0.8
[INFO] 2020-03-11 16:41:59,217 [     args.py:   70]:	decr_ratio: 0.8
2020-03-11 16:41:59,217-INFO: dev_set: dataset/SST-2/dev.tsv
[INFO] 2020-03-11 16:41

In [18]:
!python3 predict_classifier.py -u --ernie_config_path script/en_glue/ernie_base/ernie_config.json --init_check_point checkpoints/

python3: can't open file 'predict_classifier.py': [Errno 2] No such file or directory
